# import libraries

In [12]:
import json
import re
import os
from PIL import Image
import matplotlib.pyplot as plt
from C_D_create_embeddings import embed_caption_api

# query embeddings

In [ ]:
# Initialize a dictionary to store embeddings for each column
query_emb_dict = {
    'Specific Category': [],
    'Color': [],
    'Material': [],
    'Features': [],
    'Style': [],
    'Occasion': []
}


# Loop through each pandas column (subembed_key) and get embeddings for each entry
for subembed_key in query_emb_dict.keys():
    text = query_emb_dict[subembed_key] # Initialize/reset the list for each column
    if text:
        query_emb_dict[subembed_key].append(embed_caption_api([text]))
    


# calculate similarity score

In [ ]:

# search for the best match between query and database embeddings

def norm_embeddings(embeds):
    norms = torch.norm(embeds, dim=1, keepdim=True)
    normalized_embeddings = embeds / norms
    return normalized_embeddings

def torchify_norm(emb_dict):
    new_dict = {}
    for col, val in emb_dict.items():
        torch_val = torch.tensor(val)
        if val:
            new_dict[col] = norm_embeddings(torch_val)
    return new_dict

query_emb_dict_torch = torchify_norm(query_emb_dict)
# To Do: load embeddings from the database
embeddings_dict_torch = torchify_norm(embeddings_dict)


def search_match(query_emb_dict_torch, embeddings_dict_torch, score_weights):
    N = embeddings_dict_torch["Color"].shape[0]
    weighted_sim = torch.zeros((1, N))
    for col, val in query_emb_dict_torch.items(): 
        cosine_sim_matrix = torch.mm(val, embeddings_dict_torch[col].t()) # or use @
        weighted_sim +=  cosine_sim_matrix * score_weights[col]      
    return weighted_sim

scores = search_match(query_emb_dict_torch, embeddings_dict_torch, score_weights)
# show the best matching 3 images

# visualize top 3 best matching images

In [ ]:
output_dir = "C:/Users/Esra/Desktop/Deep_Learning/Image_Classification/Fashion/Classify_ThreadUp_Images/data/StanfordJpegs/dedupped_images/"

top_ix = scores.topk(3).indices.tolist()[0]
# Iterate over the responses and display each image with its description
for item in top_ix:
    
    img_path = os.path.join(output_dir, captions["filename"][item])
    
    # Open the image
    img = Image.open(img_path)

    # Set up the matplotlib figure
    plt.figure(figsize=(6, 6))

    # Display the image
    plt.imshow(img)
    plt.axis('off')  # Remove axis ticks

    # Display the description
    plt.title(captions["description"][item], wrap=True)

    # Show the plot
    plt.show()